<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/03-RAG_with_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index==0.9.21 openai==1.6.0 cohere==4.39 tiktoken==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probabilit

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"

In [5]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-dataset.json

--2023-12-25 17:33:36--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25361 (25K) [text/plain]
Saving to: ‘mini-dataset.json’

mini-dataset.json   100%[===================>]  24.77K  --.-KB/s    in 0.006s  

2023-12-25 17:33:37 (3.76 MB/s) - ‘mini-dataset.json’ saved [25361/25361]



### Read JSON

In [15]:
import json

with open('./mini-dataset.json', 'r') as file:
    data = json.load(file)

In [16]:
len( data['chunks'] )

22

In [18]:
texts = [item['text'] for item in data['chunks']]

### Apply Embedding

In [24]:
from llama_index import Document

documents = [Document(text=t) for t in texts]

In [25]:
from llama_index import VectorStoreIndex

# build index / generate embeddings using OpenAI
index = VectorStoreIndex.from_documents(documents)

In [29]:
# Save the generated embeddings.
# index.storage_context.persist(persist_dir="indexes")

### Query Dataset

In [27]:
query_engine = index.as_query_engine()

In [28]:
response = query_engine.query(
    "How many parameters LLaMA2 model has?"
)
print(response)

The Llama 2 model has four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.
